In [1]:
from langchain_openai import ChatOpenAI

from extract import (
    extract_function_names,
    extract_function_by_name,
    extract_imports,
)

from llm_calls import (
    llm_generate_unit_tests,
    llm_generate_function_from_unit_tests,
    llm_find_bug,
    llm_fix_bug,
)

from run_tests import (
    find_first_failing_unit_test,
)

from function_code_with_unit_tests import FunctionCodeWithUnitTests

llm = ChatOpenAI(temperature=0, model="gpt-4o")

query = "Extract emails (including instances of 'name at email dot com') from a text."

In [2]:
unit_test_code_as_string = llm_generate_unit_tests(llm, query)
print(unit_test_code_as_string)

# Function to be tested: extract_emails

import re
import pytest

def test_extract_emails_basic():
    text = "Contact us at support@example.com for more info."
    expected = ["support@example.com"]
    assert extract_emails(text) == expected

def test_extract_emails_multiple():
    text = "Emails: first@example.com, second@example.org, third@example.net"
    expected = ["first@example.com", "second@example.org", "third@example.net"]
    assert extract_emails(text) == expected

def test_extract_emails_with_name_at_email_dot_com():
    text = "Please email john at example dot com for details."
    expected = ["john@example.com"]
    assert extract_emails(text) == expected

def test_extract_emails_mixed_formats():
    text = "Reach out at jane.doe@example.com or jane at example dot com."
    expected = ["jane.doe@example.com", "jane@example.com"]
    assert extract_emails(text) == expected

def test_extract_emails_no_emails():
    text = "There are no emails in this text."
    expected 

In [3]:
unit_test_function_names = extract_function_names(code_string=unit_test_code_as_string)
unit_test_function_names

['test_extract_emails_basic',
 'test_extract_emails_multiple',
 'test_extract_emails_with_name_at_email_dot_com',
 'test_extract_emails_mixed_formats',
 'test_extract_emails_no_emails',
 'test_extract_emails_invalid_emails',
 'test_extract_emails_edge_case',
 'test_extract_emails_with_special_characters',
 'test_extract_emails_with_unicode',
 'test_extract_emails_case_sensitivity',
 'test_extract_emails_multiline',
 'test_extract_emails_overlapping',
 'test_extract_emails_with_lookahead',
 'test_extract_emails_with_lookbehind',
 'test_extract_emails_greedy_vs_nongreedy',
 'test_extract_emails_with_anchors',
 'test_extract_emails_with_flags',
 'test_extract_emails_escape_sequences',
 'test_extract_emails_large_input',
 'test_extract_emails_empty_input']

In [4]:
unit_test_imports = extract_imports(unit_test_code_as_string)
unit_test_imports

['import re', 'import pytest']

In [5]:
target_function_code_as_string = llm_generate_function_from_unit_tests(
    llm, unit_test_code_as_string
)
print(target_function_code_as_string)

import re
from typing import List

def extract_emails(text: str) -> List[str]:
    """
    Extracts email addresses from the given text. Supports standard email formats
    and some common obfuscated formats like 'name at domain dot com'.

    Parameters:
    text (str): The input text from which to extract email addresses.

    Returns:
    List[str]: A list of extracted email addresses.
    """
    # Regular expression to match standard email formats
    email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    # Regular expression to match obfuscated email formats like 'name at domain dot com'
    obfuscated_pattern = re.compile(r'\b([A-Za-z0-9._%+-]+)\s+at\s+([A-Za-z0-9.-]+)\s+dot\s+([A-Z|a-z]{2,})\b')

    # Find all standard email addresses
    emails = email_pattern.findall(text)
    
    # Find all obfuscated email addresses and convert them to standard format
    obfuscated_emails = obfuscated_pattern.findall(text)
    for obfuscated in obfuscated_

In [37]:
function_code_with_unit_tests = FunctionCodeWithUnitTests(
    function_code=[target_function_code_as_string],
    unit_tests={
        name: [
            extract_function_by_name(
                code_string=unit_test_code_as_string, function_name=name
            )
        ]
        for name in unit_test_function_names
    },
    unit_test_imports="\n".join(unit_test_imports)
)


In [38]:
latest_unit_test_to_fix = ""

for i in range(10):

    first_failing_unit_test_message = find_first_failing_unit_test(
        function_code_with_unit_tests=function_code_with_unit_tests
    )
    print("")
    first_failing_unit_test_message.print_message_details()
    print("")

    if first_failing_unit_test_message.all_tests_pass:
        break

    # If it causes an error two times in a row, delete the unit test
    if latest_unit_test_to_fix == first_failing_unit_test_message.unit_test_name:
        unit_test_code_to_delete = function_code_with_unit_tests.get_unit_test_code(
            first_failing_unit_test_message.unit_test_name
        )
        print(
            f"Unable to fix unit test:\n{unit_test_code_to_delete}It will be deleted\n"
        )
        function_code_with_unit_tests.delete_unit_test_code(
            first_failing_unit_test_message.unit_test_name
        )
        continue

    latest_unit_test_to_fix = first_failing_unit_test_message.unit_test_name

    print("llm find bug call")
    bug_report_dict = llm_find_bug(
        llm=llm,
        function_code_with_unit_tests=function_code_with_unit_tests,
        first_failing_unit_test_message=first_failing_unit_test_message,
    )

    if bug_report_dict["Likely Bug Location"] == "Unit Test":
        print("llm fix unit test bug call")
        updated_unit_test_code = llm_fix_bug(
            llm=llm,
            bug_report_dict=bug_report_dict,
            function_code_with_unit_tests=function_code_with_unit_tests,
            first_failing_unit_test_message=first_failing_unit_test_message,
        )
        function_code_with_unit_tests.replace_unit_test(
            unit_test_name=first_failing_unit_test_message.unit_test_name,
            new_unit_test_code=updated_unit_test_code,
        )
    elif bug_report_dict["Likely Bug Location"] == "Function":
        print("llm fix function bug call")
        updated_function_code = llm_fix_bug(
            llm=llm,
            bug_report_dict=bug_report_dict,
            function_code_with_unit_tests=function_code_with_unit_tests,
            first_failing_unit_test_message=first_failing_unit_test_message,
        )
        function_code_with_unit_tests.replace_function_code(
            new_function_code=updated_function_code,
        )
    else:
        raise ValueError("bug_report_dict keys not as expected")


print("All tests are passing")


all_tests_pass: False
unit_test_name: test_extract_emails_edge_case
output: Assertion failed: 

llm find bug call
llm fix function bug call

all_tests_pass: False
unit_test_name: test_extract_emails_with_unicode
output: Assertion failed: 

llm find bug call
llm fix function bug call

all_tests_pass: False
unit_test_name: test_extract_emails_with_unicode
output: Assertion failed: 

Unable to fix unit test:
def test_extract_emails_with_unicode():
    text = 'Unicode emails: üser@example.com, 用户@例子.广告'
    expected = ['üser@example.com', '用户@例子.广告']
    assert extract_emails(text) == expected
It will be deleted


all_tests_pass: False
unit_test_name: test_extract_emails_overlapping
output: Assertion failed: 

llm find bug call
llm fix function bug call

all_tests_pass: False
unit_test_name: test_extract_emails_overlapping
output: Assertion failed: 

Unable to fix unit test:
def test_extract_emails_overlapping():
    text = 'Overlapping: user@example.comuser@example.com'
    expected = ['

In [31]:
print(function_code_with_unit_tests.get_function_code())

import re
from typing import List

def extract_emails(text: str) -> List[str]:
    """
    Extracts email addresses from the given text. Supports standard email formats
    and some common obfuscated formats like 'name at domain dot com'.

    Parameters:
    text (str): The input text from which to extract email addresses.

    Returns:
    List[str]: A list of extracted email addresses.
    """
    # Regular expression to match standard email formats, including Unicode characters
    email_pattern = re.compile(r'\b[\w.%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{1,}\b', re.UNICODE)
    # Regular expression to match obfuscated email formats like 'name at domain dot com'
    obfuscated_pattern = re.compile(r'\b([\w.%+-]+)\s+at\s+([A-Za-z0-9.-]+)\s+dot\s+([A-Z|a-z]{1,})\b', re.UNICODE)

    # Find all standard email addresses
    emails = email_pattern.findall(text)
    
    # Find all obfuscated email addresses and convert them to standard format
    obfuscated_emails = obfuscated_pattern.findall(t

In [32]:
for unit_test_name in function_code_with_unit_tests.unit_tests:
    print(
        function_code_with_unit_tests.get_unit_test_code(unit_test_name=unit_test_name)
    )

def test_extract_emails_basic():
    text = 'Contact us at support@example.com for more info.'
    expected = ['support@example.com']
    assert extract_emails(text) == expected

def test_extract_emails_multiple():
    text = 'Emails: first@example.com, second@example.org, third@example.net'
    expected = ['first@example.com', 'second@example.org', 'third@example.net']
    assert extract_emails(text) == expected

def test_extract_emails_with_name_at_email_dot_com():
    text = 'Please email john at example dot com for details.'
    expected = ['john@example.com']
    assert extract_emails(text) == expected

def test_extract_emails_mixed_formats():
    text = 'Reach out at jane.doe@example.com or jane at example dot com.'
    expected = ['jane.doe@example.com', 'jane@example.com']
    assert extract_emails(text) == expected

def test_extract_emails_no_emails():
    text = 'There are no emails in this text.'
    expected = []
    assert extract_emails(text) == expected

def test_extract

Before the fix
```
import re
from typing import List

def extract_emails(text: str) -> List[str]:
    """
    Extracts email addresses from the given text. Supports standard email formats
    and some common obfuscated formats like 'name at domain dot com'.

    Parameters:
    text (str): The input text from which to extract email addresses.

    Returns:
    List[str]: A list of extracted email addresses.
    """
    # Regular expression to match standard email formats
    email_pattern = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
    # Regular expression to match obfuscated email formats like 'name at domain dot com'
    obfuscated_pattern = re.compile(r'\b([A-Za-z0-9._%+-]+)\s+at\s+([A-Za-z0-9.-]+)\s+dot\s+([A-Z|a-z]{2,})\b')

    # Find all standard email addresses
    emails = email_pattern.findall(text)
    
    # Find all obfuscated email addresses and convert them to standard format
    obfuscated_emails = obfuscated_pattern.findall(text)
    for obfuscated in obfuscated_emails:
        emails.append(f"{obfuscated[0]}@{obfuscated[1]}.{obfuscated[2]}")

    return emails
```

After the fix
```
import re
from typing import List

def extract_emails(text: str) -> List[str]:
    """
    Extracts email addresses from the given text. Supports standard email formats
    and some common obfuscated formats like 'name at domain dot com'.

    Parameters:
    text (str): The input text from which to extract email addresses.

    Returns:
    List[str]: A list of extracted email addresses.
    """
    # Regular expression to match standard email formats, including Unicode characters
    email_pattern = re.compile(r'\b[\w.%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{1,}\b', re.UNICODE)
    # Regular expression to match obfuscated email formats like 'name at domain dot com'
    obfuscated_pattern = re.compile(r'\b([\w.%+-]+)\s+at\s+([A-Za-z0-9.-]+)\s+dot\s+([A-Z|a-z]{1,})\b', re.UNICODE)

    # Find all standard email addresses
    emails = email_pattern.findall(text)
    
    # Find all obfuscated email addresses and convert them to standard format
    obfuscated_emails = obfuscated_pattern.findall(text)
    for obfuscated in obfuscated_emails:
        emails.append(f"{obfuscated[0]}@{obfuscated[1]}.{obfuscated[2]}")

    return emails
```


Before the fix
```
def test_extract_emails_overlapping():
    text = "Overlapping: user@example.comuser@example.com"
    expected = ["user@example.com", "user@example.com"]
    assert extract_emails(text) == expected
```

After the fix
```
def test_extract_emails_overlapping():
    text = 'Overlapping: user@example.com, user@example.com'
    expected = ['user@example.com', 'user@example.com']
    assert extract_emails(text) == expected
```